# hddm_model_estimate_check: model5

In [1]:
import hddm
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os,sys
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib as mpl
import seaborn as sns
#print 'HDDM version:',hddm.__version__

In [2]:
data=hddm.load_csv('2.UAhddm/fMRI_sessionData_DDM.csv')
data=data.rename(columns={'subj':'subj_idx','RT':'rt','islie':'response','session':'ses'})
data

,Unnamed: 0,subj_idx,trial,round,menu,menu_position_1,menu_position_2,true_answer,condition,reinforcer,...,rt,choose,response,ses,money_diff,former_lie,former_honest,consis_diff,diff,former_diff
0,0,101,1,1,1,1,37,1,enhance_honesty,4,...,1.551243,1,0,1,100,0,0,0,-4,0
1,1,101,2,1,25,25,61,25,enhance_honesty,2,...,1.775196,25,0,1,100,0,0,0,-2,0
2,2,101,3,1,6,6,42,6,enhance_random,6,...,1.734819,6,0,1,100,0,0,0,-6,0
3,3,101,4,1,2,38,2,2,enhance_lie,8,...,2.359468,38,1,1,100,0,0,0,8,0
4,4,101,5,1,9,45,9,9,enhance_random,6,...,1.625446,9,0,1,100,0,0,0,-6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10651,283,143,68,8,32,32,68,32,enhance_lie,2,...,1.413876,32,0,4,100,0,7,-7,2,-7
10652,284,143,69,8,27,63,27,27,enhance_random,8,...,0.796888,27,0,4,100,1,6,-5,-8,-5
10653,285,143,70,8,30,66,30,30,enhance_random,6,...,1.154773,30,0,4,100,0,7,-7,-6,-7
10654,286,143,71,8,11,47,11,11,enhance_lie,2,...,1.238054,11,0,4,100,0,7,-7,2,-7


In [3]:
###drop RT that are too fast###

rt_thresh_lb = 0.300 #300 ms lower threshold
rt_thresh_ub = 4
group_data = data[data['rt'] > rt_thresh_lb]
group_data = group_data[group_data['rt'] < rt_thresh_ub]

In [4]:
#Create result directory
reg_model_num = 5
date_var = '2.UAhddm/DDM_result' 
if os.path.isdir(date_var) == False:
    os.mkdir(date_var)    


    
group_analysis_dir = os.path.join(date_var,'Group')
if os.path.isdir(group_analysis_dir) == False:
    os.mkdir(group_analysis_dir)

group_reg1_analysis_dir = os.path.join(group_analysis_dir,'GroupReg'+str(reg_model_num))
if os.path.isdir(group_reg1_analysis_dir) == False:
    os.mkdir(group_reg1_analysis_dir)
group_reg1_posterior_analysis_dir = os.path.join(group_reg1_analysis_dir,'Posteriors')
if os.path.isdir(group_reg1_posterior_analysis_dir) == False:
    os.mkdir(group_reg1_posterior_analysis_dir)
 

In [5]:
len(pd.unique(data.subj_idx))

37

# Estimate model

In [6]:

# model1
num_samples = 100000
num_burn = 50000
num_thin=25
num_subs = len(pd.unique(data.subj_idx))

v_reg = 'v ~  1 + former_diff*ses + diff*ses'
a_reg = 'a ~ 1  + ses'
z_reg = 'z ~ 1 + ses'
reg = [v_reg, a_reg, z_reg]
model_group_reg1 = hddm.HDDMRegressor(data,reg,include=['v','a','z'],group_only_regressors=False,informative=False)
model_group_reg1_traces = os.path.join(group_reg1_analysis_dir,'traces_reg'+str(reg_model_num)+'.db')

model_group_reg1.sample(num_samples,burn=num_burn,thin=num_thin,dbname=model_group_reg1_traces,db='pickle')

#print#

model_group_reg1_name = 'stats_print_all_model_group_n' + str(num_subs) + '_reg' + str(reg_model_num) + '.csv'
model_group_reg1_out_path = os.path.join(group_reg1_analysis_dir,model_group_reg1_name)
model_group_reg1.print_stats(fname=model_group_reg1_out_path)
#stats#
model_group_reg1_stats = model_group_reg1.gen_stats()
model_group_reg1_stats_name = 'stats_all_group_n' + str(num_subs) + '_reg' + str(reg_model_num) + '_stats.csv'
model_group_reg1_stats_out_path = os.path.join(group_reg1_analysis_dir,model_group_reg1_stats_name)
model_group_reg1_stats.to_csv(model_group_reg1_stats_out_path)
dic_reg1 = model_group_reg1.dic

#save reg1
model_group_reg1_sname = 'model_group_reg' + str(reg_model_num) + '_n' + str(num_subs)
model_group_reg1_path = os.path.join(group_reg1_analysis_dir,model_group_reg1_sname)
model_group_reg1.save(model_group_reg1_path)

##save posterior plots for reg1##
model_group_reg1.plot_posteriors(save=True,path=group_reg1_posterior_analysis_dir)
plt.close('all')



No model attribute --> setting up standard HDDM
Set model to ddm


/opt/conda/lib/python3.8/site-packages/hddm/models/base.py:1307: UserWarning:  
 Your include statement misses either the v, a or t parameters. 
Parameters not explicitly included will be set to the defaults, 
which you can find in the model_config dictionary!
  warnings.warn(


 [-----------------100%-----------------] 100001 of 100000 complete in 172480.0 sechddm sampling elpased time:  172480.457 s
Plotting v_Intercept
Plotting v_Intercept_std
Plotting v_former_diff
Plotting v_former_diff_std
Plotting v_ses
Plotting v_ses_std
Plotting v_former_diff:ses
Plotting v_former_diff:ses_std
Plotting v_diff
Plotting v_diff_std
Plotting v_diff:ses
Plotting v_diff:ses_std
Plotting a_Intercept
Plotting a_Intercept_std
Plotting a_ses
Plotting a_ses_std
Plotting z_Intercept
Plotting z_Intercept_std
Plotting z_ses
Plotting z_ses_std


In [7]:
model_group_reg1.dic

28159.06416895621

# Post-predictive check

In [9]:
num_subs=37
model_group_reg1_sname = 'model_group_reg' + str(reg_model_num) + '_n' + str(num_subs)
model_group_reg1_path = os.path.join(group_reg1_analysis_dir,model_group_reg1_sname)
model_group_reg1=hddm.load(model_group_reg1_path)

In [12]:
ppc_data = hddm.utils.post_pred_gen(model_group_reg1)
ppc_compare = hddm.utils.post_pred_stats(data, ppc_data)
ppc_compare

Start generating posterior prediction...


/opt/conda/lib/python3.8/site-packages/kabuki/analyze.py:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_stats[name][i] = value
/opt/conda/lib/python3.8/site-packages/kabuki/analyze.py:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_stats[name][i] = value
/opt/conda/lib/python3.8/site-packages/kabuki/analyze.py:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_stats[name][i] = value
/opt/conda/lib/python3

,observed,mean,std,SEM,MSE,credible,quantile,mahalanobis
stat,,,,,,,,
accuracy,0.302459,0.253002,0.133961,0.002446,0.020392,True,67.416216,0.369190
mean_ub,1.710363,2.148017,0.907351,0.191541,1.014826,True,34.261065,0.482342
std_ub,1.416620,2.162120,0.989363,0.555771,1.534610,True,23.553743,0.753515
10q_ub,0.832484,0.510808,0.271255,0.103475,0.177054,True,90.320865,1.185882
30q_ub,1.173008,0.892041,0.414566,0.078942,0.250807,True,76.771258,0.677737
50q_ub,1.468738,1.404752,0.628394,0.004094,0.398973,True,58.653903,0.101825
70q_ub,1.824811,2.295576,1.015121,0.221620,1.252090,True,35.088341,0.463753
90q_ub,2.660930,4.642860,2.111040,3.928044,8.384534,True,16.066891,0.938840
mean_lb,-1.482836,-1.922151,0.587971,0.192998,0.538707,True,73.875676,0.747172


In [13]:
pd.DataFrame(ppc_compare).to_csv(group_reg1_analysis_dir+'/ppc'+str(reg_model_num) + '.csv')

# Test of convergence

In [ ]:

from kabuki.analyze import gelman_rubin


v_reg = 'v ~  1 + former_diff*ses + diff*ses'
a_reg = 'a ~ 1  + ses'
z_reg = 'z ~ 1 + ses'
reg = [v_reg, a_reg, z_reg]

models = []
for i in range(5):
    m = hddm.HDDMRegressor(data,reg,group_only_regressors=False,include=['v','a','z'],informative=False)
    m.find_starting_values()
    m.sample(15000, burn=5000)
    models.append(m)

gelman_rubin(models)

No model attribute --> setting up standard HDDM
Set model to ddm


/opt/conda/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2309: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-----------------100%-----------------] 15001 of 15000 complete in 38712.1 sechddm sampling elpased time:  38712.942 s
No model attribute --> setting up standard HDDM
Set model to ddm


/opt/conda/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2309: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-----------------100%-----------------] 15001 of 15000 complete in 39310.7 sechddm sampling elpased time:  39311.958 s
No model attribute --> setting up standard HDDM
Set model to ddm


...

 [-----------------100%-----------------] 15001 of 15000 complete in 40692.4 sechddm sampling elpased time:  40693.326 s
No model attribute --> setting up standard HDDM
Set model to ddm


/opt/conda/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2309: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-----------------100%-----------------] 15001 of 15000 complete in 39927.3 sechddm sampling elpased time:  39928.137 s
No model attribute --> setting up standard HDDM
Set model to ddm


/opt/conda/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2309: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-----            14%                  ] 2113 of 15000 complete in 5592.9 sec

In [ ]:
pd.DataFrame(gelman_rubin(models),index=[0]).T.to_csv(group_reg1_analysis_dir+'/coverg'+str(reg_model_num) + '.csv')